## Data Exploration - 120 years of Olympics

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_athletes = pd.read_csv('./data/athlete_events.csv')

In [ ]:
print('Number of entries {}'.format(df_athletes.shape[0]))
print('Features:')
for x in list(df_athletes.columns):
    print('\t *\t{}'.format(x))

In [ ]:
df_athletes.head()

### Countries with more medals

In [ ]:
medals = df_athletes.dropna().groupby(['Sex', 'NOC', 'Medal']).count().reset_index()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(20,5), sharey=True)
for idx, med in enumerate(['Gold', 'Silver', 'Bronze']):
    comitee = pd.pivot_table(medals[medals.Medal == med], index='NOC', columns='Sex').fillna(0)
    comitee[('ID', 'Total')] = comitee[('ID', 'F')] + comitee[('ID', 'M')]
    comitee.sort_values([('ID', 'Total')]).iloc[-20:,:2].plot(kind='bar', stacked=True, ax=ax[idx], title='Comitee with highest number of {} medals'.format(med))

#### Summer Games

In [ ]:
medals_summer = df_athletes[df_athletes['Season'] == 'Summer'].dropna().groupby(['Sex', 'NOC', 'Medal'])\
                .count().reset_index()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(20,5), sharey=True)
for idx, med in enumerate(['Gold', 'Silver', 'Bronze']):
    comitee = pd.pivot_table(medals_summer[medals_summer.Medal == med], index='NOC', columns='Sex').fillna(0)
    comitee[('ID', 'Total')] = comitee[('ID', 'F')] + comitee[('ID', 'M')]
    comitee.sort_values([('ID', 'Total')]).iloc[-20:,:2].plot(kind='bar', stacked=True, ax=ax[idx], title='Comitee with highest number of {} medals'.format(med))

#### Winter Games

In [ ]:
medals_winter = df_athletes[df_athletes['Season'] == 'Winter'].dropna().groupby(['Sex', 'NOC', 'Medal'])\
                .count().reset_index()

In [ ]:
f, ax = plt.subplots(1,3, figsize=(20,5), sharey=True)
for idx, med in enumerate(['Gold', 'Silver', 'Bronze']):
    comitee = pd.pivot_table(medals_winter[medals_winter.Medal == med], index='NOC', columns='Sex').fillna(0)
    comitee[('ID', 'Total')] = comitee[('ID', 'F')] + comitee[('ID', 'M')]
    comitee.sort_values([('ID', 'Total')]).iloc[-20:,:2].plot(kind='bar', stacked=True, ax=ax[idx], title='Comitee with highest number of {} medals'.format(med))

### Number of athletes per year

In [ ]:
df_athletes['type_event'] = df_athletes['Event'].apply(lambda x: ' '.join((x.split()[2:])))

#### Summer Games

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_athletes[df_athletes.Season == 'Summer'].groupby('Year').agg({'Name':'nunique'}).\
plot(kind='bar', ax=ax, title='Number of athletes by year')
ax.set_xlabel('Year')
ax.set_ylabel('Number of athletes')
plt.show()

In [ ]:
f, ax = plt.subplots(2, 1, figsize=(18,12))
pd.pivot_table(df_athletes[df_athletes.Season == 'Summer'].groupby(['Year', 'Sex']).agg({'Name':'count'}), \
              index='Year', columns='Sex').\
plot(kind='bar', stacked=True, title='Number of athletes by year - Summer', ax=ax[0])
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Number of athletes')
ax[0].legend(['Female', 'Male'])

pd.pivot_table(df_athletes[df_athletes.Season == 'Winter'].groupby(['Year', 'Sex']).agg({'Name':'count'}), \
              index='Year', columns='Sex').\
plot(kind='bar', stacked=True, title='Number of athletes by year - Winter', ax=ax[1])
ax[1].set_xlabel('Year')
ax[1].set_ylabel('Number of athletes')
ax[1].legend(['Female', 'Male'])
plt.show()

In [ ]:
print('Number of unique athletes: {}'.format(df_athletes.Name.nunique()))
print('Number of unique athletes (Summer): {}'.format(\
        df_athletes[df_athletes.Season == 'Summer'].Name.nunique()))
print('Number of unique athletes (Winter): {}'.format(\
        df_athletes[df_athletes.Season == 'Winter'].Name.nunique()))
print('Number of unique host cities: {}'.format(df_athletes.City.nunique()))
print('Number of unique host cities (Summer): {}'.format(\
        df_athletes[df_athletes.Season == 'Summer'].City.nunique()))
print('Number of unique host cities (Winter): {}'.format(\
        df_athletes[df_athletes.Season == 'Winter'].City.nunique()))

In [ ]:
print('Athletes participating in both summer and winter games: ')
a = set(df_athletes[df_athletes.Season == 'Summer'].Name.unique())
b = set(df_athletes[df_athletes.Season == 'Winter'].Name.unique())
print(len(a.intersection(b)))
df_athletes[df_athletes.Name.isin(list(a.intersection(b)))].Sport.unique()

### Host cities

In [ ]:
df_cities = pd.read_csv('./data/host_cities.csv')
df_cities.head()

## Summer Olympics

### Sports by year

In [ ]:
sport_by_year = df_athletes[df_athletes.Season == 'Summer'][['Year', 'Sport']].groupby('Year')['Sport'].apply(set)

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_athletes[df_athletes.Season == 'Summer'].groupby('Year').agg({'Sport':'nunique'}).\
plot(kind='bar', ax=ax, title='Number of sports by year')
ax.set_xlabel('Year')
ax.set_ylabel('Number of different sports')
plt.show()

In [ ]:
years = sorted(df_athletes[df_athletes.Season == 'Summer'].Year.unique())
disappearing_sports = [(years[i], list(sport_by_year[years[i-1]] - sport_by_year[years[i]])) for i in range(1,len(years))]
new_sports = [(years[i], list(sport_by_year[years[i]] - sport_by_year[years[i-1]])) for i in range(1,len(years))]

In [ ]:
# adding the first year
new_sports.insert(0, (years[0], list(sport_by_year[years[0]])))
disappearing_sports.insert(0, (years[0], []))

In [ ]:
for e, d in zip(new_sports, disappearing_sports):
    print('Year: ', e[0])
    print(' New sports: ', e[1])
    print('Disappearing sports:')
    print(d[1],'\n')

In [ ]:
pd.DataFrame([(x,y[1],z[1]) for x, y, z in zip(years, new_sports, disappearing_sports)], columns=['Year', 'New', 'Disappearing'])

In [ ]:
sport_year = pd.pivot_table(df_athletes[df_athletes.Season == 'Summer'][['Year', 'Sport', 'ID']].groupby(['Year', 'Sport'])\
               .count().reset_index(),
               index='Sport', columns='Year').fillna(0)

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_athletes[df_athletes.Season == 'Summer'][['Year', 'Sport']].groupby(['Sport'])\
               .nunique().sort_values('Year').plot(kind='bar',y='Year', ax=ax)
plt.show()

In [ ]:
# Winter sports in the summer games
df_athletes[(df_athletes.Season == 'Summer') & \
            (df_athletes.Sport.isin(df_athletes[df_athletes.Season == 'Winter'].Sport.unique()))].\
groupby(['Sport', 'Year']).nunique()

### Sports

In [ ]:
sports_category = df_athletes[df_athletes['Season']=='Summer'].groupby('Sport').nunique()['Event'].sort_values(ascending=False)
sports_category

We can see that there are sports with a lot of events while there are others with very few of them. This can clearly influence on the number of medals, i.e. the same athlete can win many medals in different events of swimming while in cricket there is only one event.

In [ ]:
sport_event = df_athletes[df_athletes['Medal']=='Gold'].groupby(['Event', 'Year'], as_index=False).\
                count()[['Event', 'Medal']].drop_duplicates(subset=['Event'], keep='last')

In [ ]:
data_sports = []

for _, row in sport_event.iterrows():
    if 'Men' in row['Event']:
        gender = 'M'
    elif 'Women' in row['Event']:
        gender = 'W'
    else:
        gender = 'B'
    if row['Medal'] > 1:
        type_sport = 'Team'
    else:
        type_sport = 'Individual'
    data_sports.append([row['Event'], type_sport, gender])

In [ ]:
df_events = pd.DataFrame(data_sports, columns=['Event', 'Collectivity', 'Gender'])
df_events.head()

In [ ]:
df_events.groupby('Collectivity').count()['Event'].plot(kind='bar', figsize=(10,4))

In [ ]:
# B - both/mixed, M - Men, W - Women
df_events.groupby('Gender').count()['Event'].plot(kind='bar', figsize=(10,4))

In [ ]:
df_athletes = df_athletes.join(df_events.set_index('Event'), on='Event', how='left')

In [ ]:
sports_type = df_athletes[df_athletes['Season'] == 'Summer'].\
        groupby(['Sport', 'Collectivity']).\
        nunique()['Event']

In [ ]:
sports_type

In [ ]:
sports_type_unstack = sports_type.unstack().fillna(0)
sports_type_unstack.assign(tmp=sports_type_unstack.sum(axis=1)).\
        sort_values('tmp').drop('tmp', 1).\
        plot(kind='bar', stacked=True, figsize=(20,12));

### Countries by year

In [ ]:
country_per_year = df_athletes[df_athletes.Season == 'Summer'].groupby(['Year'])['NOC'].nunique()

f, ax = plt.subplots(figsize=(20,5))
country_per_year.plot(kind='bar', title='Number of countries per year', ax=ax)
ax.set_xlabel('Year')
ax.set_ylabel('Number of different countries')
plt.show();

In [ ]:
years = sorted(df_athletes[df_athletes.Season == 'Summer'].Year.unique())
noc_per_year = df_athletes[df_athletes.Season == 'Summer'][['Year', 'NOC']].groupby('Year')['NOC'].apply(set)

disappearing_noc = [(years[i], list(noc_per_year[years[i-1]] - noc_per_year[years[i]])) for i in range(1,len(years))]
new_noc = [(years[i], list(noc_per_year[years[i]] - noc_per_year[years[i-1]])) for i in range(1,len(years))]

In [ ]:
# adding the first year
new_noc.insert(0, (years[0], list(noc_per_year[years[0]])))
disappearing_noc.insert(0, (years[0], []))

In [ ]:
df_noc = pd.read_csv('./data/noc_regions.csv')
df_noc.head()

In [ ]:
# create dictionary with NOC and regions
noc_dict = df_noc[['NOC', 'region']].set_index('NOC').to_dict()['region']

# add missing entry
noc_dict['SGP'] = 'Singapore'

In [ ]:
for e, d in zip(new_noc, disappearing_noc):
    print('Year: ', e[0])
    print('\n\t New country (NOC): ', [noc_dict[i] for i in e[1]])
    print('\n\t Disappearing country (NOC):', [noc_dict[i] for i in d[1]],'\n\n')

In [ ]:
noc_years = df_athletes[df_athletes.Season == 'Summer'].groupby(['NOC']).nunique()['Year']\
            .sort_values(ascending=False)
noc_years

### Host cities

In [ ]:
df_host_cities = df_athletes[['City', 'Year']].drop_duplicates().reset_index(drop=True)

In [ ]:
df_host_cities = df_host_cities.join(df_cities.set_index('City'), on='City')

In [ ]:
df_host_cities

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_host_cities[df_host_cities['Season'] == 'Summer'].groupby('Country')['City']\
    .count().sort_values().plot(kind='bar', ax=ax, title='Years helding the Summer Olympic Games')
ax.set_ylabel('Years helding the Games')
ax.set_yticks(range(0,5))
plt.show()

In [ ]:
f, ax = plt.subplots(figsize=(20,5))
df_host_cities[df_host_cities['Season'] == 'Summer'].groupby('Continent')['City']\
    .count().sort_values().plot(kind='bar', ax=ax, title='Years helding the Summer Olympic Games')
ax.set_ylabel('Years helding the Games')
ax.set_yticks(range(0,19,2))
plt.show()

## Athletes 

### Who has more medals?

In [ ]:
count_medals = df_athletes.groupby(['Name', 'Medal']).agg({'Sport':'count'}).\
                rename(columns={'Sport':'total'}).reset_index()

**Greatest number of GOLD MEDALS**

In [ ]:
count_medals[count_medals.Medal == 'Gold'].sort_values('total', ascending=False).head(10)

**Greatest number of SILVER MEDALS**

In [ ]:
count_medals[count_medals.Medal == 'Silver'].sort_values('total', ascending=False).head(10)

**Greatest number of BRONZE MEDALS**

In [ ]:
count_medals[count_medals.Medal == 'Bronze'].sort_values('total', ascending=False).head(10)

**Greatest number of MEDALS**

In [ ]:
count_medals.groupby('Name').sum().sort_values('total', ascending=False).head(10)

### Who has participated more years?

In [ ]:
ath_year_sport = df_athletes.groupby(['Name']).agg({
    'Year':'nunique', 'Sport':'unique'
})

In [ ]:
ath_year_sport.sort_values(['Year', 'Name'], ascending=False).head(15)

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('Year', ascending=False).reset_index().iloc[0,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('Year', ascending=False).reset_index().iloc[1,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('Year', ascending=False).reset_index().iloc[2,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

### Who has participated in greater number of different sports?

In [ ]:
ath_year_sport['number_Sports'] = ath_year_sport['Sport'].apply(lambda x: len(x))

In [ ]:
ath_year_sport.sort_values('number_Sports', ascending=False).head(10)

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('number_Sports', ascending=False).reset_index().iloc[0,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('number_Sports', ascending=False).reset_index().iloc[1,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

In [ ]:
df_athletes[df_athletes.Name == ath_year_sport.sort_values('number_Sports', ascending=False).reset_index().iloc[2,0]].\
groupby(['Name','Year', 'Season', 'Sport']).agg({
    'type_event':'unique', 'Medal':['unique','nunique']
})

### Who has taken part in both Summer and Winter Olympics?

In [ ]:
df_athletes[df_athletes.Name.isin(list(a.intersection(b)))].groupby('Name').agg({
    'Sport':['unique', 'nunique'], 'Team':['unique'], 'Year':['unique', 'nunique'], 'Medal':['unique']
}).sort_values(('Sport','nunique'), ascending=False)

## History of the Games

In [ ]:
df_athletes[['Year','Season']].drop_duplicates().sort_values('Year').reset_index(drop=True)

#### General Observations

* The Olympic Games are celebrated every 4 years. **What happened in 1906?** The **1906 Intercalated Games were held in Athens.** These Games were, but are not now, officially recognised by the IOC and no Intercalated Games have been held since. 


* In **1924 the first Winter Olympic Games** were held in Chamonix. However, Figure skating (in 1908 and 1920) and Ice hockey (in 1920) were featured as Olympic events at the Summer Olympics.


* Both Summer and Winter Olympic Games were celebrated the same year until 1992. Beginning in 1994, the Winter Olympics were held every four years, two years after each Summer Olympics.


* In **1916 no Games were held**. They were cancelled due to the outbreak of World War I. 


* In **1940 and 1944 no Games were held**. They were cancelled due to the World War II.


* The next Games were supposed to be held in **2020** in Tokyo, Japan. However, due to the coronavirus pandemic, **the Games will be rescheduled** to a date no later than summer 2021, and that the Games would keep the name Tokyo 2020. This is the first time in history that the Olympics have been postponed, rather than canceled.